<a href="https://colab.research.google.com/github/irispoonyt/textanalysis/blob/main/product_process_22Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!apt update
!apt-get install libopencv-dev

[Errno 2] No such file or directory: 'darknet'
/content
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cr

In [ ]:
#import libraries
import re 
import string
import pandas as pd
import numpy as np
import glob
import copy
import collections
import io

import seaborn as sns
from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim

#gensim
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


#spacy
import spacy

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from nltk import tokenize 
from operator import itemgetter
import math 

import matplotlib


# NLTK Resource
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') # for tagging
nltk.download('stopwords') # stopwords

#vis 
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/AIdata/onlyfamily_id_en.csv", sep = ';',header=0, decimal=",")
# df1 = pd.read_csv("/content/drive/MyDrive/Iris/AIdata/onlyfamily_id_en.csv", sep = ';',header=0, decimal=",")
df1

,appln_id,subapplications,applications,docdb_family_id,appln_auth,appln_filing_date,appln_filing_year,docdb_family_size,appln_nr_original,appln_abstract,language
0,21675509,Physiological parameter monitoring,Life and medical sciences,1725797,GB,31/12/9999,9999,1,NaN,"132,307. Barry, A. G., and Proudfoot, A. Aug. ...",en
1,15593310,Genetics/genomics,Life and medical sciences,3497097,WO,30/03/2000,2000,2,EP0002784,The invention relates to a method for prognosi...,en
2,13443450,Genetics/genomics,Life and medical sciences,3516369,DE,21/09/1996,1996,2,19638853.0,The method for formulating a model based on th...,en
3,855830,Medical sciences,Life and medical sciences,3688388,WO,08/10/2002,2002,3,AT0200289,The invention relates to a device and a method...,en
4,49722486,Neuroscience/neurorobotics,Life and medical sciences,3697785,US,06/01/1982,1982,1,337397,An evoked response audiometer is disclosed in ...,en
...,...,...,...,...,...,...,...,...,...,...,...
75096,554839453,Bioinformatics,Life and medical sciences,76991704,WO,22/01/2021,2021,1,US2021/014684,The present invention provides phase separatio...,en
75097,554839485,Public health,Life and medical sciences,76991711,WO,22/01/2021,2021,1,US2021/014755,The present disclosure provides methods for co...,en
75098,554843213,Medical informatics,Life and medical sciences,76992646,WO,22/01/2021,2021,1,US2021/014754,"Disclosed herein are systems, devices, and met...",en
75099,554844654,Neuroscience/neurorobotics,Life and medical sciences,76992923,WO,19/01/2021,2021,1,US2021/014003,Provided herein are magnetic resonance imaging...,en


In [ ]:
# selected column headers we want to use 
new_headers = ['appln_id', 'appln_abstract']

In [ ]:
# remove unwanted columns keep selected headers
abstract = pd.DataFrame(df1, columns=new_headers)

abstract

,appln_id,appln_abstract
0,21675509,"132,307. Barry, A. G., and Proudfoot, A. Aug. ..."
1,15593310,The invention relates to a method for prognosi...
2,13443450,The method for formulating a model based on th...
3,855830,The invention relates to a device and a method...
4,49722486,An evoked response audiometer is disclosed in ...
...,...,...
75096,554839453,The present invention provides phase separatio...
75097,554839485,The present disclosure provides methods for co...
75098,554843213,"Disclosed herein are systems, devices, and met..."
75099,554844654,Provided herein are magnetic resonance imaging...


In [ ]:
# check data type
abstract.dtypes

appln_id           int64
appln_abstract    object
dtype: object

In [ ]:
# word count are min 1 to max 175 

# Identify most common words
common = pd.Series(' '.join(abstract['appln_abstract']).split()).value_counts()[:40]
common

In [ ]:
#Identify uncommon words
uncommon =  pd.Series(' '.join(abstract
         ['appln_abstract']).split()).value_counts()[-40:]
uncommon

In [ ]:
#Cleaning the data
# convert to lowercase
abstract['appln_abstract'] = abstract['appln_abstract'].str.lower()

# convert desc to string
abstract['appln_abstract'] = abstract['appln_abstract'].astype(str)

abstract.dtypes

build list A and list B

In [ ]:
# Defining a list of dictionaries in Python 
list_process = ['process','method','algorithm', 'processing','detection','analysis','control','monitoring','provides', 'step', 
                    'generating', 'evaluating', 'optimal','efficient','acquisition','identification','calculating','receive','server','characteristic','subsequent','support','framework','point','sensor','watch','screen','mode','screening','extraction','procedure','workflow','automatic','continous','assembling','building','compose','construct',
                    'cast','coat','cut','fabricating','form','install','making','manufacture','mold','prepare','shape','spray','decompose',
                    'breed','model','training-sample','delivery','organise','organize','sort','corresponding-system','corresponding-program','corresponding-stage',
                'configure','system']
  
dictOfprocess = { i : 1 for i in list_process}
#dictOfprocess = dict.fromkeys(list_process , 1)

# Printing the results
print(dictOfprocess)
 
# Validating the type of 'dictOfprocess' and its element
print(type(dictOfprocess))

In [ ]:
# Defining a list of dictionaries in Python 
list_product = ['product','connected','feature','function','device','equipment','effective','terminal','type','attribute','multi','multifunctional',
                'realize','accuracy','replacement','adjustment','instrument','tool', 'additional','broaden','extend','broaden-the-range',
                'medium','platform','frame','application','specificity','circuit','discovery','results','various','apparatus','solution','means',
                 ]
  
dictOfproduct = { i : 1 for i in list_product}
#dictOfprocess = dict.fromkeys(list_process , 1)

# Printing the results
print(dictOfproduct)
 
# Validating the type of 'dictOfprocess' and its element
print(type(dictOfproduct))

In [ ]:
#creating the stop word list and adding to it
# library for text preprocessing
import re
import nltk

#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')   # Required for tokenization
nltk.download('wordnet') # Required for lemmatization

#nltk.download('wordnet') 


In [ ]:
# create a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))

# create a list of custom stopwords
new_words =  ['The','a', 'of', 'and', 'to', 'is', 'the', 'in', 'for', 'an', 'on', 'image', 'with', 'data', 'by', 'are', 'be', 
              'or', 'can', 'that', 'invention', 'based', 'medical', 'from', 'as', 'at', 'information', 'one', 
              'comprises','first', 'which', 'device', 'through', 'each', 'using', 'used', 'according', 'sciences', 'data', 'according', 'patient', 'time', 'network',
              'using','body', 'user', 'used', 'unit', 'discloses', 'second', 'set', 'provided that','provided with','images', 'signal', 
              'plurality', 'least', 'value', 'us', 'target', 'includes','real','de','result','corresponding','diagnosis','may','obtain','parameter',
              'may','obtain','utility','region','determining','part','field','obtained','present','treatment','neural','training',
              'high','parameters', 'disease','wherein','sensor','computer','end','area','improved','arranged','learning','structure',
              'dimensional','obtaining','input','state','sample','display','operation','genetics','genomics','optimization','output',
              'storage','following','power','subject','including','test','condition','position','physiological','performing','different',
              'segmentation','human','side','surface','prediction','heart','point','object','use','plate','cell','pressure','signals','comprise','problem','values',
              'tissue','informatics','box','rate','sequence','associated','layer','number','technical','selected','nursing','measurement','intelligent','level',
              'current','individual','temperature','reference','brain','frequency','said','energy', 'whether', 'clinical','jp','digital',
              'light','face','low','electronic','cancer', 'within', 'vessel', 'risk','pixel', 'upper','biological', 'ct', 'fixed',
              'population','medicine','respectively','protein','degree','threshold','p','pulse','embodiment','2011','original','water',
              'person','28', 'and', 'as','(ãâ¢ãâãâpetãâ¢ãâãâ)','g'
]
stop_words = stop_words.union(new_words)

stop_words

In [ ]:
#remove useless words/ stopwords
text1 = abstract['appln_abstract'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

text1.head(50)

In [ ]:
text1=text1.astype(str)


In [ ]:
# Instantiate
lemmatizer = WordNetLemmatizer()

In [ ]:
# Exclusion list of punctuations and numbers
exclist = string.punctuation + string.digits
# Print the exclusion list
print(exclist)

First, we will tokenize first before removing numbers and punctuation, so that the hypenated words can be kept

In [ ]:
import nltk
nltk.download('omw-1.4')

# Create a Function
def clean_texts(text1):
    """ Function to perform preprocessing """

    
    # Tokenization
    tokens = word_tokenize(text1)
        
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    

    # Join tokens
    clean_text = " ".join(tokens)

   
   # Remove punctuations and numbers
    text1 = text1.translate(str.maketrans("", "", exclist))
  
    # Return the output
    return text1
# Apply the function to all disclosures
text1 = text1.apply(clean_texts)
# View the first 5 rows
text1.head()

In [ ]:
print(text1)

In [ ]:
print(abstract)

In [ ]:
!pip install rake-nltk -q
!pip install python-rake==1.4.4

In [ ]:
from rake_nltk import Rake

In [ ]:
import RAKE
import operator

In [ ]:
print(type(stop_words))

In [ ]:
stop_words1=list(stop_words)

In [ ]:
print(type(stop_words))

In [ ]:
rake_object=RAKE.Rake(stop_words1)

In [ ]:
def Sort_Tuple(tup):
  #reverse=None(Sorts in Ascending order)
  #key is set to sort using second element of
  #sublist lambda has been used
  tup.sort(key=lambda x: x[1])
  return tup

In [ ]:
#convert object to string
text2=str(text1)

In [ ]:
r = Rake(punctuations = [')','(',',',':','),',').','.'])

In [ ]:
r=Rake()
r.extract_keywords_from_text(text2)
r.get_ranked_phrases()

In [ ]:
#double-checking
# word_score = degree(word)/frequency(word)r.extract_keywords_from_text(text[100])

In [ ]:
keywords=r.extract_keywords_from_text(text1[100])
extracted_keyword=r.get_ranked_phrases()
print(extracted_keyword)

In [ ]:
r.get_ranked_phrases()

In [ ]:
len(extracted_keyword)

In [ ]:
# word_score = degree(word)/frequency(word)
phrase_df = pd.DataFrame(r.get_ranked_phrases_with_scores(), columns = ['score','phrase'])
phrase_df.loc[phrase_df.score>3]

Keywords extraction (total)

In [ ]:
# Python3 code to demonstrate working of
# Extract Keywords from String List
 
# Using iskeyword() + loop + split()
import keyword
 
 # Create a Function
def clean_texts(text1):
    """ Function to perform preprocessing """

    
    # Tokenization
    tokens = word_tokenize(text1)
        
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    

    # Join tokens
    clean_text = " ".join(tokens)

   # Remove punctuations and numbers
    text1 = text1.translate(str.maketrans("", "", exclist))
    
    # Return the output
    return clean_text
# Apply the function to all disclosures
text1 = text1.apply(clean_texts)

# iterating using loop
res = []
for sub in text1:
   for word in sub.split():
 
       # check for keyword using iskeyword()
       if keyword.iskeyword(word):
           res.append(word)

patentkw=str(res)
 
# printing result
print("Extracted Keywords : " + patentkw)

In [ ]:
len(patentkw)

In [ ]:
dictOfpatent = { i : 1 for i in patentkw}

In [ ]:
abstract.head()

In [ ]:
#on a macro-level, which keyword is more commonly found is list A and list B, respectively

In [ ]:
#count number of items in the list/ length of a list
#printing result
length_process=len(dictOfprocess)
length_product=len(dictOfproduct)
print("The length of process list is :"+str(length_process))
print("The length of product list is :"+str(length_product))


#length of process=64; whereas length of product=34
#Since list A and list B are of different lengths, so I need to calculate each pair of keywords in list A and list B, 
#calculate the occurrence by counting or mutual information, Then I will get a size of list A x size of list B
#a model with measure of tendency is necessary to run regression on the i-th patent
#average over list PMI (keywords in patent abstract, keywords in list A and list B)


In [ ]:
#for each row of abstract keywords, build a cooccurence matrix with List A and List B


In [ ]:


import numpy as np
import itertools


def by_indexes(iterable):
    output = {}
    for index, key in enumerate(iterable):
        output.setdefault(key, []).append(index)
    return output


def co_occurrence_matrix(text1, dictOfprocess, window_size=5):
    def split_tokens(tokens):
        for token in tokens:
            indexs = dictOfprocess_indexes.get(token)
            if indexs is not None:
                yield token, indexs[0]

    matrix1 = np.zeros((len(dictOfprocess), len(dictOfprocess)), np.float64)
    dictOfprocess_indexes = by_indexes(dictOfprocess)

    for sent in text1:
        tokens = by_indexes(split_tokens(sent.split())).items()
        for ((word_1, x), indexes_1), ((word_2, y), indexes_2) in itertools.permutations(tokens, 2):
            for k in indexes_1:
                for l in indexes_2:
                    if abs(l - k) <= window_size:
                        matrix1[x, y] += 1
    return matrix1

process_matrix=co_occurrence_matrix(text1, dictOfprocess)

print(process_matrix)

In [ ]:
import numpy as np
import itertools


def by_indexes(iterable):
    output = {}
    for index, key in enumerate(iterable):
        output.setdefault(key, []).append(index)
    return output


def co_occurrence_matrix(text1, dictOfproduct, window_size=5):
    def split_tokens(tokens):
        for token in tokens:
            indexs = dictOfproduct_indexes.get(token)
            if indexs is not None:
                yield token, indexs[0]

    matrix2 = np.zeros((len(dictOfproduct), len(dictOfproduct)), np.float64)
    dictOfproduct_indexes = by_indexes(dictOfproduct)

    for sent in text1:
        tokens = by_indexes(split_tokens(sent.split())).items()
        for ((word_1, x), indexes_1), ((word_2, y), indexes_2) in itertools.permutations(tokens, 2):
            for k in indexes_1:
                for l in indexes_2:
                    if abs(l - k) <= window_size:
                        matrix2[x, y] += 1
    return matrix2

product_matrix=co_occurrence_matrix(text1, dictOfproduct)

print(product_matrix)

In [ ]:
plt.imshow(process_matrix,interpolation='nearest', cmap='Reds')
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(product_matrix,interpolation='nearest', cmap='Reds')
plt.colorbar()
plt.show()

The graph suggests that the patent is more product-oriented

In [ ]:
patentkw1=list(patentkw)

In [ ]:
#co-ocurrence matrix: study overall correlation
#mutual information between an abstract and a list of dictionary
#gives you some information about closeness in the vector space
#1. calculate which keyword is most found among all abstracts, list A and list B
 
# Most common keyword in abstract and list process


In [ ]:
#2. Which one appears more on list A but not list B (or the other way round)

In [ ]:
#Part 1, both occures
#Part 2: occur list A but not list B
#Part 3: occur list B but not list ]\
#Part 4: both not found

Example

In [ ]:
#check the electronyque machine - case study
!pip3 install keybert

In [ ]:
from keybert import KeyBERT
example= """A seizure detection system including one or more circuits, the one or more circuits configured to receive an electroencephalogram (EEG) signal generated based on electrical brain activity of a patient. The one or more circuits are configured to determine metrics based on the EEG signal, the metrics indicating non-linear features of the EEG signal, determine that the EEG signal indicates a candidate seizure by determining, based at least in part on the metrics, a change in the non-linear features of the EEG signal over time, and generate a seizure alert indicating that the EEG signal indicates the candidate seizure. The change in the non-linear features indicates a physiological force that gives rise to the candidate seizure."""

kw_model = KeyBERT()
keywords3 = kw_model.extract_keywords(example)
print(keywords3)

In [ ]:
len(keywords3)

In [ ]:
coocur_matrix = np.zeros((len(keywords3), len(list_process)),np.float64)

25 Nov calculate PMI for each patent

In [ ]:
text3=pd.DataFrame(text1) #<---data frame version
print(text3)

In [ ]:
#Convert the dictionary into DataFrame
dfprocess=pd.Series(dictOfprocess)
dfprocess=pd.Series(dictOfproduct)

In [ ]:
print(dfprocess)

In [ ]:
print(pmi)

In [ ]:
ListA = "respiratory analysis system ( 101 ) monitoring system"
#dictOfA=dict(ListA,False)
dfA=pd.Series(ListA)

In [ ]:
print (dfA)

In [ ]:
#practise PMI score 28 Nov 2022

from glob import glob
import nltk
from nltk import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
from collections import Counter
from collections import OrderedDict

from keybert import KeyBERT
import itertools
import numpy as np
np.random.seed(3) # for reproducibility


# for tokenization
import spacy

# for log2
import math
from math import log

# for bigrams
from nltk import bigrams

 # Create a Function
def clean_texts(dfA):
    """ Function to perform preprocessing """
#convert it to a token

    tokens = nltk.word_tokenize(dfA)
        
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove punctuations and numbers
    dfA= dfA.translate(str.maketrans("", "", exclist))
    

    # Join tokens
    clean_text = " ".join(tokens)


    return dfA
    
dfA1=dfA.apply(clean_texts)

import re
dfA2=dfA1.to_string(index=False)
keywordfA = re.findall(r'\w+',dfA2) #it is a list
print(keywordfA)

#kw_model = KeyBERT()
#keywordfA = kw_model.extract_keywords(dfA1)

# printing result
print("Extracted Keywords : " + str(keywordfA)) #<------extraction of keywords keywordfA= keywords for patent


# initializing dict to store frequency of each element
#for patent keyword
elements_count = {}
# iterating over the elements for frequency
for element in keywordfA:
   # checking whether it is in the dict or not
   if element in elements_count:
      # incerementing the count by 1
      elements_count[element] += 1
   else:
      # setting the count to 1
      elements_count[element] = 1
# printing the elements frequencies
for key, value in elements_count.items():
   print(f"{key}: {value}")

   
   countdfA=elements_count.items()

countprocess=dictOfprocess.items()

print(countprocess)
#make a list of unique items in keyword list
unique_valuesA=set(keywordfA) #avoid duplications

# generate combinations for each sub list seperately  
list=[unique_valuesA, list_process] #avoid duplications 

combination = [p for p in itertools.product(*list)]  #compute bigrams for keywords in patent and process list

combination_diagonal = np.diagonal(combination) 

#total lengths of the list/ total items of the list

total_combination=len(combination)

totalA=len(keywordfA)

length_process=length_process



#prob of word in abstract
for item in keywordfA:
    c = Counter(keywordfA)
    totalA = sum(c.values())
    percentA = {key: value/totalA for key, value in c.items()}
#    percentA = {key: value/totalA for key, value in c.items()}
    print(percentA)

    # convert to list
    percent_listA = [percentA.get(str(i), 0.0) for i in range(5)]
    #    percent_listA = [percentA.get(str(i), 0.0) for i in range(5)]
    print(percent_listA)


for item in list_process:
    d = Counter(list_process)
    totalprocess = sum(d.values())
    percentprocess = {key: value/totalprocess for key, value in d.items()}
    print(percentprocess)

    # convert to list
    percent_listprocess = [percentprocess.get(str(i), 0.0) for i in range(5)]
    print(percent_listprocess)

#calculate probabilities of co-occurence
for item in combination:
    e = Counter(combination)
    totalcomb = sum(e.values())
    percent_cooccurence = {key: value/totalcomb for key, value in e.items()}
    print(percent_cooccurence)

    # convert to list
    percent_listcooccurence = [percent_cooccurence.get(str(i), 0.0) for i in range(5)]
    print(percent_listcooccurence)

#calculate pmi




for (f,g,h) in zip(percentA, percentprocess,percent_cooccurence):
    f = Counter(percentA)
    g = Counter(percentprocess)
    h = Counter(percent_cooccurence)

def calculate_pmi(cooccurrence, prob_word1, prob_word2):
    """
    Calcualtes the pmi of 2 words
    """
    
    return math.log2(cooccurrence / (prob_word1 * prob_word2))


# A list storing all the pmi values
all_pmi = []


for k, v in h.items():
    # k = tuple, e.g. ('respiratoty, control')
    # v = prob of cooccurrence, e.g. 0.15625
    
    # Extract the two words from the tuple
    word1 = k[0]
    word2 = k[1]
    #print(word1, word2)
    
    # Get the probability of the word in f and g
    prob_word1 = f.get(word1)
    prob_word2 = g.get(word2)
    
 
    pmi = calculate_pmi(v, prob_word1, prob_word2)
    all_pmi.append(pmi)
    print(f"Word 1: {word1}, Word 2: {word2}, prob(Word 1 in f): {prob_word1}, prob(Word 2 in g): {prob_word2}, pmi = {pmi}")
    
    
# Calculate the mean of all the pmi values
pmi_mean = sum(all_pmi) / len(all_pmi)
print(f"Mean of all the pmi values = {pmi_mean}")


In [ ]:
print(f"Mean of all the pmi values = {pmi_mean}")

In [ ]:
#Loop for every column (every patent abstract) + process
#1 Dec 2022
from glob import glob
import nltk
from nltk import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
from collections import Counter
from collections import OrderedDict

from keybert import KeyBERT
import itertools
import numpy as np
np.random.seed(3) # for reproducibility
import re


# for tokenization
import spacy

# for log2
import math
from math import log

# for bigrams
from nltk import bigrams

# Create a Function

def clean_texts(text1):
  
  tokens2 = nltk.word_tokenize(text1)   #convert it to a token
  
  tokens2 = [lemmatizer.lemmatize(token) for token in tokens2] # Lemmatization
  
  text1= text1.translate(str.maketrans("", "", exclist))# Remove punctuations and numbers
  
  clean_text = " ".join(tokens2)# Join tokens
  
  return text1


for ind in text1:

  text4 = text1.apply(clean_texts(ind))
# text5=text4.to_string(index=False)


  keyword_pt = re.findall(r'\W+',text4) #it is a list
#print(keyword_pt)

'''
#break here
#kw_model = KeyBERT()
#keywordfA = kw_model.extract_keywords(dfA1)

# printing result
print("Extracted Keywords : " + str(keyword_pt)) #<------extraction of keywords keywordfA= keywords for patent


# initializing dict to store frequency of each element
#for patent keyword
elements_count = {}
# iterating over the elements for frequency
for element in keyword_pt:
   # checking whether it is in the dict or not
   if element in elements_count:
      # incerementing the count by 1
      elements_count[element] += 1
   else:
      # setting the count to 1
      elements_count[element] = 1
# printing the elements frequencies
for key, value in elements_count.items():
  elements=f"{key}:{value}"
  #print(f"{key}: {value}")
  countpt=elements_count.items()
  
  countprocess=dictOfprocess.items()

print(countprocess)
#make a list of unique items in keyword list
unique_valuespt=set(keyword_pt) #avoid duplications

# generate combinations for each sub list seperately  
list1=[unique_valuespt, list_process] #avoid duplications 

combination_process = [p for p in itertools.product(*list1)]  #compute bigrams for keywords in patent and process list

combination_diagonalx = np.diagonal(combination_process) 

#total lengths of the list/ total items of the list

total_combinationx=len(combination_process)

totalpt=len(keyword_pt)

length_process=length_process



#prob of word in abstract
for item in keyword_pt:
    c = Counter(keyword_pt)
    totalpt = sum(c.values())
    percentpt = {key: value/totalpt for key, value in c.items()}

#   print(percentpt)

    # convert to list
    percent_listpt = [percentpt.get(str(i), 0.0) for i in range(5)]

#    print(percent_listA)


for item in list_process:
    d = Counter(list_process)
    totalprocess = sum(d.values())
    percentprocess = {key: value/totalprocess for key, value in d.items()}
    print(percentprocess)

    # convert to list
    percent_listprocess = [percentprocess.get(str(i), 0.0) for i in range(5)]
    print(percent_listprocess)

#calculate probabilities of co-occurence
for item in combination_process:
    e = Counter(combination_process)
    total_combx = sum(e.values())
    percent_cooccurence = {key: value/totalcombx for key, value in e.items()}
    print(percent_cooccurence)

    # convert to list
    percent_listcooccurence = [percent_cooccurence.get(str(i), 0.0) for i in range(5)]
    print(percent_listcooccurence)

#calculate pmi



for (f,g,h) in zip(percentpt, percentprocess,percent_cooccurence):
    f = Counter(percentpt)
    g = Counter(percentprocess)
    h = Counter(percent_cooccurence)

def calculate_pmi(cooccurrence, prob_word1, prob_word2):
    """
    Calcualtes the pmi of 2 words
    """
    
    return math.log2(cooccurrence / (prob_word1 * prob_word2))


# A list storing all the pmi values
all_pmi = []


for k, v in h.items():
    # k = tuple, e.g. ('respiratoty, control')
    # v = prob of cooccurrence, e.g. 0.15625
    
    # Extract the two words from the tuple
    word1 = k[0]
    word2 = k[1]
    #print(word1, word2)
    
    # Get the probability of the word in f and g
    prob_word1 = f.get(word1)
    prob_word2 = g.get(word2)
    
 
    pmi = calculate_pmi(v, prob_word1, prob_word2)
    all_pmi.append(pmi)
    print(f"Word 1: {word1}, Word 2: {word2}, prob(Word 1 in f): {prob_word1}, prob(Word 2 in g): {prob_word2}, pmi = {pmi}")
    
    
# Calculate the mean of all the pmi values
pmi_mean = sum(all_pmi) / len(all_pmi)
print(f"Mean of all the pmi values = {pmi_mean}")'''

In [ ]:
def clean_texts(text1):
  
  tokens2 = nltk.word_tokenize(text1)   #convert it to a token
  
  tokens2 = [lemmatizer.lemmatize(token) for token in tokens2] # Lemmatization
  
  text1= text1.translate(str.maketrans("", "", exclist))# Remove punctuations and numbers
  
  clean_text = " ".join(tokens2)# Join tokens
  
  return text1

#text4=text3.to_string(index=False)

for ind in text1:
  print(clean_texts(ind))

In [ ]:
print

In [ ]:
 # Create a Function

def clean_texts(text1):
  
  tokens = nltk.word_tokenize(text1) # Join tokens
  
  tokens = [lemmatizer.lemmatize(token) for token in tokens] # Lemmatization
  
  text1= text1.translate(str.maketrans("", "", exclist))    # Remove punctuations and numbers
  
  clean_text = " ".join(tokens)# Join tokens
  
  return text1

text4=text1.astype(str)
#text4=text1.to_string(index=False)

for row in text1.T.iteritems():
  
  text4=text1.apply(clean_texts(row))
  
  keyword_pt= re.findall(r'\w+',text4) #it is a list
'''  print(keyword_pt)'''

In [ ]:
print(text1)

In [ ]:
print(keyword_pt[0:30])

In [ ]:
#Loop for every column (every patent abstract) + product
'''
from glob import glob
import nltk
from nltk import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
from collections import Counter
from collections import OrderedDict

from keybert import KeyBERT
import itertools
import numpy as np
np.random.seed(3) # for reproducibility


# for tokenization
import spacy

# for log2
import math
from math import log

# for bigrams
from nltk import bigrams

 # Create a Function

for ind in text3.index:
  
  def clean_texts(text3):
    """ Function to perform preprocessing """
#convert it to a token

    tokens = nltk.word_tokenize(text3)
        
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove punctuations and numbers
    text3= text3.translate(str.maketrans("", "", exclist))
    

    # Join tokens
    clean_text = " ".join(tokens)


    return text3

    
text4 = text3.apply(clean_texts)

#import re
#text41=text4.to_string(index=False)
#keyword_pt = re.findall(r'\w+',text41) #it is a list
#print(keyword_pt)

#kw_model = KeyBERT()
#keywordfA = kw_model.extract_keywords(dfA1)

# printing result
#print("Extracted Keywords : " + str(keyword_pt)) #<------extraction of keywords keywordfA= keywords for patent


# initializing dict to store frequency of each element
#for patent keyword
elements_count = {}
# iterating over the elements for frequency
for element in keyword_pt:
   # checking whether it is in the dict or not
   if element in elements_count:
      # incerementing the count by 1
      elements_count[element] += 1
   else:
      # setting the count to 1
      elements_count[element] = 1
# printing the elements frequencies
for key, value in elements_count.items():
   print(f"{key}: {value}")

   
   countpt=elements_count.items()'''

countproduct=dictOfproduct.items()

print(countproduct)
#make a list of unique items in keyword list
unique_valuespt=set(keyword_pt) #avoid duplications

# generate combinations for each sub list seperately  
list2=[unique_valuespt, list_product] #avoid duplications 

combination_product = [p for p in itertools.product(*list2)]  #compute bigrams for keywords in patent and process list

combination_diagonaly = np.diagonal(combination_product) 

#total lengths of the list/ total items of the list

total_combinationy=len(combination_product)

totalpt=len(keyword_pt)

length_product=length_product



#prob of word in abstract
for item in keyword_pt:
    c = Counter(keyword_pt)
    totalpt = sum(c.values())
    percentpt = {key: value/totalpt for key, value in c.items()}

#   print(percentpt)

    # convert to list
    percent_listpt = [percentpt.get(str(i), 0.0) for i in range(5)]

#    print(percent_listA)


for item in list_product:
    d = Counter(list_product)
    totalproduct = sum(d.values())
    percentproduct = {key: value/totalproduct for key, value in d.items()}
    print(percentproduct)

    # convert to list
    percent_listproduct = [percentproduct.get(str(i), 0.0) for i in range(5)]
    print(percent_listproduct)

#calculate probabilities of co-occurence
for item in combination_product:
    e = Counter(combination_product)
    totalcomby = sum(e.values())
    percent_cooccurence = {key: value/totalcomby for key, value in e.items()}
    print(percent_cooccurence)

    # convert to list
    percent_listcooccurence = [percent_cooccurence.get(str(i), 0.0) for i in range(5)]
    print(percent_listcooccurence)

#calculate pmi




for (f,g,h) in zip(percentpt, percentprocess,percent_cooccurence):
    f = Counter(percentpt)
    g = Counter(percentprocess)
    h = Counter(percent_cooccurence)

def calculate_pmi(cooccurrence, prob_word1, prob_word2):
    """
    Calcualtes the pmi of 2 words
    """
    
    return math.log2(cooccurrence / (prob_word1 * prob_word2))


for k, v in h.items():
    # k = tuple, e.g. ('respiratoty, control')
    # v = prob of cooccurrence, e.g. 0.15625
    
    # Extract the two words from the tuple
    word1 = k[0]
    word2 = k[1]
    #print(word1, word2)
    
    # Get the probability of the wor in f and g
    prob_word1 = f.get(word1)
    prob_word2 = g.get(word2)
    
 
    pmi_product = calculate_pmi(v, prob_word1, prob_word2)
    print(f"Word 1: {word1}, Word 2: {word2}, prob(Word 1 in f): {prob_word1}, prob(Word 2 in g): {prob_word2}, pmi = {pmi_product}")



In [ ]:
print(type(text3))

In [ ]:
print(text3)


In [ ]:
  def clean_texts(text3):
    """ Function to perform preprocessing """
#convert it to a token

    tokens = nltk.word_tokenize(text3)
        
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove punctuations and numbers
    text3= text3.translate(str.maketrans("", "", exclist))
    

    # Join tokens
    clean_text = " ".join(tokens)


    return text3

    
text4=text3.apply(clean_texts)

import re
text41=text4.to_string(index=False)
keyword_pt = re.findall(r'\w+',text41) #it is a list
print(keyword_pt[30])


In [ ]:
sns.set_theme()

fig, ax = plt.subplots(figsize=(8,8))

sns.scatterplot(data=df, x='pmi_process', y='pmi_product', ax=ax)

plt.title('product and process matrix')

plt.xlabel('Product')

plt.ylabel('Process')

In [ ]:
len(combination)

In [ ]:
# generate combinations for each sub list seperately
lists_of_pairs = [list(itertools.combinations(sub_list, 2)) for sub_list in names]
# flatten the lists of pairs to 1 large list of pairs
all_pairs = [pair for pairs_list in lists_of_pairs for pair in pairs_list]
# let the Counter do the rest for you
co_occurences_counts = Counter(all_pairs)


#compute co-occurence
def listOccurences(item, names):
    # item is the list that you want to check, eg. ['cat','fish']
    # names contain the list of list you have.
    set_of_items = set(item) # set(['cat','fish'])
    count = 0
    for value in names:
        if set_of_items & set(value) == set_of_items:
            count+=1
    return count

names =  [['cat', 'fish'], ['cat'], ['fish', 'dog', 'cat'],['cat', 'bird', 'fish'], ['fish', 'bird']]
# Now for each of your possibilities which you can generate
# Chain flattens the list, set removes duplicates, and combinations generates all possible pairs.
permuted_values = list(itertools.combinations(set(itertools.chain.from_iterable(names)), 2))
d = {}
for v in permuted_values:
    d[str(v)] = listOccurences(v, names)
# The key in the dict being a list cannot be possible unless it's converted to a string.
print(d)
# {"['fish', 'dog']": 1, "['cat', 'dog']": 1, "['cat', 'fish']": 3, "['cat', 'bird']": 1, "['fish', 'bird']": 2}



#create co-occurence matrix


#compute PMI score


def pmi(df):
    '''
    Calculate the positive pointwise mutal information score for each entry
    https://en.wikipedia.org/wiki/Pointwise_mutual_information
    We use the log( p(y|x)/p(y) ), y being the column, x being the row
    '''
    # Get numpy array from pandas df
    dfA2 = dfA1.to_numpy()
    dfprocess1=dfprocess.to_numpy()

 
    # Get numpy array from pandas text1 and list process
    arr = text1.as_matrix()
    brr=list_process.as_matrix()


    # p(y|x) probability of each t1 overlap within the row
    row_totals = arr.sum(axis=1).astype(float)
    prob_cols_given_row = (arr.T / row_totals).T

    # p(y) probability of each t1 in the total set
    col_totals = arr.sum(axis=0).astype(float)
    prob_of_cols = col_totals / sum(col_totals)

    # PMI: log( p(y|x) / p(y) )
    # This is the same data, normalized
    ratio = prob_cols_given_row / prob_of_cols
    ratio[ratio==0] = 0.00001
    _pmi = np.log(ratio)
    _pmi[_pmi < 0] = 0

    return _pmi
    return _pmi
    